In [1]:
from pythreejs import *
from IPython.display import display
from math import pi
import numpy as np

This test examines the inner workings of the Geometry classes
Geometries should be creatable via the helper extras/Geometries classes
as well as procedurally creating the vertices yourself

In [2]:
g = BoxGeometry(
    width=5, 
    height=10, 
    depth=15,
    widthSegments=5, 
    heightSegments=10,
    depthSegments=15)

In [3]:
g

Preview(child=BoxGeometry(depth=15.0, depthSegments=15, height=10.0, heightSegments=10, width=5.0, widthSegmen…

In [4]:
g2 = Geometry.from_geometry(g)

In [5]:
np.shape(g2.vertices), len(g2.faces)

((0,), 0)

In [6]:
g2

Preview(child=Geometry(colors=['#ffffff']), shadowMap=WebGLShadowMap())

In [7]:
g3 = Geometry(vertices=g2.vertices[:], faces=g2.faces[:])

In [8]:
g3

Preview(child=Geometry(colors=['#ffffff']), shadowMap=WebGLShadowMap())

In [9]:
g2.faces

()

In [10]:

vertices = []
vertices.append([-50, 50, 0])
vertices.append([50, -50, 0])
vertices.append([50, 50, 0])
vertices.append([50, 50, 50])
normal = Tuple((0,0,1))
vertexcolors = ['#000000', '#0000ff', '#00ff00', '#ff0000',
     '#00ffff', '#ff00ff', '#ffff00', '#ffffff']
face = Tuple((0, 1, 2, normal, vertexcolors))
faces = [[0, 1, 2], [1, 2, 3], [3, 0, 1]]
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]
g3 = Geometry(vertices=vertices, faces=faces)
g3

Preview(child=Geometry(colors=['#ffffff'], faces=((0, 1, 2, None, ('#000000', '#0000ff', '#00ff00'), None), (1…

## Importing FreeCAD

In [11]:
import sys, os

JUPYTER_REPO_PATH = "/opt/jupyter_freecad/"

sys.path.append("/opt/freecad/freecad_build/lib")
sys.path.append(JUPYTER_REPO_PATH + "Jupyter")

import FreeCAD, FreeCADGui
FreeCADGui.setupWithoutGUI()

Creating a document with objects and a scene graph to be iterated over later on.

In [12]:
from pivy import *
from pivy.gui import soqt

doc = FreeCAD.newDocument()
doc.addObject("Part::Box","Box")
doc.addObject("Part::Cylinder","Cylinder")
doc.addObject("Part::Sphere","Sphere")
doc.addObject("Part::Torus","Torus")
doc.recompute()

from pivy import coin
root = coin.SoSeparator()
for obj in doc.Objects:
    root.addChild(FreeCADGui.subgraphFromObject(obj))

Now we need to go over the scene graph and extract the edges and surfaces. First we grab the wire representation (chosing one of the switch node children. The switch node allows switching between different FreeCAD views such as Mesh, Surface etc.)

To view the scene graph in a convenient way open `FreeCAD > Tools > view scene graph`. That's how I selected the following `SoIndexedLineSet` and the corresponding `SoCoordinate3` object. The coordinates all switch children refer to are always the coordinates in the root of the object. So one level below the document root node.

In [42]:
 def so_col_to_hex(so_color):
    color = (int(so_color[0]*255), 
                  int(so_color[1]*255),
                  int(so_color[2]*255))
    hex_col = "#{0:02x}{1:02x}{2:02x}".format(color[0],
                                              color[1],
                                              color[2])
    return hex_col

def transform_indices(so_node):
    """
    Returns list of indices from pivy.coin
    scene objects 'SoIndexedLine' and 'SoIndexedFace'
    """
    faces = list(so_node.coordIndex)
    indices = []
    curr_line = []
    for i in faces:
        if i == -1:
            indices.append(curr_line)
            curr_line = []
            continue
        curr_line.append(i)
    return indices

def generate_line_vertices(line_indices, coord_vals):
    line_vertices = []
    for i in line_indices:
        line_vertices.append(coord_vals[i[0]])
        line_vertices.append(coord_vals[i[1]])
    return line_vertices

def extract_values(res_tuple):
    # The names in the following comments refer to the
    # "Name" field in FreeCAS > tools > Scene Inspector
    # The types refer to the field "Inventor Tree".
    so_coord = res_tuple[1]
    so_faces = res_tuple[0] # Type: SoBrepFaceSet
    so_shaded_material = res_tuple[2]
    coords = list(so_coord.point)

    #print(lines)
    #print(coords)
    #print(so_coord)
    #print(so_lines)
    so_shaded_color = so_shaded_material.ambientColor.getValues()[0]
    so_shaded_emissive_color = so_shaded_material.emissiveColor.getValues()[0]
    color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    emissive_color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    color = so_col_to_hex(color)
    transparency = so_shaded_material.transparency[0]

    """
    print(face_emissive_color)
    print(line_color)
    print(face_color)
    print(so_shaded_color[3] == face_transparency)
    print(list(so_shaded_material.shininess))
    print(list(so_shaded_material.emissiveColor))
    print(transparency)
    print(list(so_shaded_material.specularColor))
    print(list(so_shaded_material.diffuseColor))
    """


    coord_vals = [list(x) for x in coords]
    indices = transform_indices(so_faces)
    
    is_line = False
    if type(so_faces) is coin.SoIndexedLineSet:
        is_line = True
    else:
        if not (type(so_faces) is coin.SoIndexedFaceSet):
            raise Exception("Unsupported type of given node: {}".format(type(so_faces)))

    #print(face_indices)
    #print(coord_vals)
    return coord_vals, indices, color, transparency, is_line

def compute_normals(faces, vertices):
    """
    Returns a list of normals for
    each vertex.
    
    Input for N faces
    should be numpy array of shape (N, 3)
    and for M vertices shape (M, 3) respectively
    """
    normals = np.zeros((len(vertices), 3), dtype='float32')
    for face in faces:
        v_index_a = face[0]
        v_index_b = face[1]
        v_index_c = face[2]
        vec_a = vertices[v_index_a]
        vec_b = vertices[v_index_b]
        vec_c = vertices[v_index_c]
        vec_a_b = np.subtract(vec_b, vec_a)
        vec_a_c = np.subtract(vec_c, vec_a)
        dot_p = np.cross(vec_a_b, vec_a_c)
        for i in [v_index_a, v_index_b, v_index_c]:
            np.add(normals[i], dot_p, normals[i])
    for norm in normals:
        np.divide(norm, np.linalg.norm(norm), norm)
    return normals

def create_geometry(res_tuple, show_faces=True, show_lines=False):
    coord_vals, indices, color, transparency, is_line = extract_values(res_tuple)

    if is_line and show_lines:
        # geometry based on coin.IndexedLineSet
        geom = create_line_geom(coord_vals, indices, color)
    elif not is_line and show_faces:
        # geometry based on coin.IndexedFaceSet
        geom = create_face_geom(coord_vals, indices, color, transparency)
    else:
        return []
    return [geom]

def create_face_geom(coord_vals, face_indices, face_color, transparency):
    vertices = np.asarray(coord_vals, dtype='float32')
    faces = np.asarray(face_indices, dtype='uint16')

    normals = compute_normals(faces, vertices)
        
    faces = faces.ravel()
    vertexcolors = np.asarray([(1,0,0)]*len(coord_vals), dtype='float32')


    faceGeometry = BufferGeometry(attributes=dict(
        position=BufferAttribute(vertices),
        index=BufferAttribute(faces),
        normal=BufferAttribute(normals)
        #colors=BufferAttribute(vertexcolors)
    ))
    
    faceGeometry.exec_three_obj_method('computeFaceNormals')
    faceGeometry.exec_three_obj_method('computeVertexNormals')

    object_mesh = Mesh(
        geometry=faceGeometry,
        material=MeshPhongMaterial(color=face_color, transparency=transparency,depthTest=True, depthWrite=True, metalness=0),
        position=[0,0,0]   # Center the cube
    )
    return object_mesh

def create_line_geom(coord_vals, line_indices, line_color):
    line_vertices = generate_line_vertices(line_indices, coord_vals)
    linesgeom = Geometry(vertices=line_vertices)
    linesgeom.exec_three_obj_method('computeVertexNormals')
    lines = Line(geometry=linesgeom, 
                 material=LineBasicMaterial(linewidth=5, color=line_color), 
                 type='LinePieces')
    return lines

def bfs_traversal(node, coordinates=None, material=None, index=0, print_tree=False):
    if print_tree:
        print(str("   " * index) + str(type(node)))
    if not (type(node) is coin.SoSwitch or type(node) is coin.SoSeparator):
        return []
    coords = coordinates
    mat = material
    edge_face_set = None
    for child in node:
        if type(child) is coin.SoCoordinate3:
            coords = child
        if type(child) is coin.SoMaterial:
            mat = child
        if type(child) is coin.SoIndexedLineSet or type(child) is coin.SoIndexedFaceSet:
            edge_face_set = child
    res_children = []
    for child in node:
        res_children.extend(bfs_traversal(child, coords, index=index+1))
    if edge_face_set:
        res = [(edge_face_set, coords, mat)]
    else:
        res = []
    res.extend(res_children)
    return res

def get_line_geometries(geometries):
    new_geometries = []
    for geom in geometries:
        line_geom = EdgesGeometry(geom.geometry)
        lines = LineSegments(geometry=line_geom, 
                 material=LineBasicMaterial(linewidth=5, color='#000000'))
        new_geometries.append(lines)
    return new_geometries

def render_objects(root_node, show_line_geom=False):
    view_width = 1200
    view_height = 1200
    geometries = []
    rendered_face_set = True
    for res in bfs_traversal(root_node):
        if type(res) is coin.SoIndexedFaceSet and rendered_face_set:
            rendered_face_set = False
            continue
        elif type(res) is coin.SoIndexedFaceSet:
            rendered_face_set = True
        geom = create_geometry(res)
        if geom:
            helper = VertexNormalsHelper(geom[0])
            geom.append(helper)
        geometries.extend(geom)
    
    if show_line_geom:
        geometries = get_line_geometries(geometries)
        
    light = PointLight(color="white", position=[40,40,40], intensity=1.0, castShadow=True)
    fog = Fog(color="#3f7b9d")
    ambient_light = AmbientLight(intensity=0.5)
    camera = PerspectiveCamera(
        position=[40, 40, 40], fov=40,
        aspect=view_width/view_height)
    children = [camera, light, ambient_light]
    children.extend(geometries)
    scene = Scene(children=children)
    controls = [OrbitControls(controlling=camera)]

    renderer = Renderer(camera=camera,
                        scene=scene, controls=controls,
                        width=view_width, height=view_height)
    renderer.shadowMap.enabled = True
    return renderer

render_objects(root)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in true_divide


Renderer(camera=PerspectiveCamera(fov=40.0, position=(40.0, 40.0, 40.0), quaternion=(0.0, 0.0, 0.0, 1.0), scal…